In [1]:
import pandas as pd
import json
from IPython.display import display, Markdown

## directory where results are
CHANSEY_ROOT = "V:"
CHANSEY_DIR = f"{CHANSEY_ROOT}/experiments/lung-malignancy-fairness-shaurya"
TEAMS_DIR = "C:/Users/shaur/OneDrive - Radboudumc/Documents - Master - Shaurya Gaur/General/Malignancy-Estimation Results"

RETRAIN_SUBSETS_DIR = f"{CHANSEY_DIR}/kiran-retrain-subsets"

What to do in this notebook:

- See racial and M:B splits for each dataset and its 10 folds.
- See if we can enrich and plug in more black patient data.

In [2]:
kiran_full = pd.read_csv(f"{TEAMS_DIR}/nlst/nlst_demov4_allmodels_cal.csv")
kiran_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16077 entries, 0 to 16076
Columns: 128 entries, PatientID to NoduleType
dtypes: bool(27), float64(86), int64(11), object(4)
memory usage: 12.8+ MB


In [9]:
kiran_full['race'].value_counts(dropna=True, normalize=False)

race
1.0    15040
2.0      481
3.0      228
6.0      177
4.0       68
5.0       44
Name: count, dtype: int64

In [4]:
kiran_full.groupby('race')['label'].mean()

race
1.0    0.076130
2.0    0.118503
3.0    0.092105
4.0    0.176471
5.0    0.045455
6.0    0.056497
Name: label, dtype: float64

In [5]:
pd.DataFrame([
    kiran_full['race'].value_counts(dropna=True, normalize=True),
    kiran_full.groupby('race')['label'].mean()
])

race,1.0,2.0,3.0,6.0,4.0,5.0
proportion,0.937773,0.029991,0.014216,0.011036,0.004240,0.002743
label,0.076130,0.118503,0.092105,0.056497,0.176471,0.045455


In [10]:
def analyze_race_and_malignancy(subset_percent='70'):
    subsetdf = pd.read_csv(f"{RETRAIN_SUBSETS_DIR}/NLST_{subset_percent}_Bogdan/dataset.csv")
    subset_demos = kiran_full[kiran_full['PatientID'].isin(subsetdf['PatientID'])]
    
    getinfo = lambda df: pd.DataFrame([
        df['race'].value_counts(dropna=True, normalize=True),
        df['race'].value_counts(dropna=True, normalize=False),
        df.groupby('race')['label'].mean(),
        df.groupby('race')['diaghype'].mean()
    ])
    
    print(f"Theoretical max black proportion = {481 / len(subsetdf)}")
    
    infodf = getinfo(subset_demos)
    display(Markdown("### Overall dataset:"))
    display(infodf)

    for i in range(10):
        display(Markdown(f"#### Fold {i}"))
        
        train_fold = pd.read_csv(f"{RETRAIN_SUBSETS_DIR}/NLST_{subset_percent}_Bogdan/train{i}.csv")
        display(Markdown(f"##### train - overall malignancy = {train_fold['label'].mean()}"))
        display(getinfo(kiran_full[kiran_full['PatientID'].isin(train_fold['PatientID'])]))
        
        valid_fold = pd.read_csv(f"{RETRAIN_SUBSETS_DIR}/NLST_{subset_percent}_Bogdan/valid{i}.csv")
        display(Markdown(f"##### valid - overall malignancy = {valid_fold['label'].mean()}"))
        display(getinfo(kiran_full[kiran_full['PatientID'].isin(valid_fold['PatientID'])]))

## 70% subset

In [11]:
analyze_race_and_malignancy('70')

Theoretical max black proportion = 0.042740358983472546


### Overall dataset:

race,1.0,2.0,3.0,6.0,4.0,5.0
proportion,0.941224,0.024757,0.016475,0.010865,0.004720,0.001959
count,10569.000000,278.000000,185.000000,122.000000,53.000000,22.000000
label,0.076450,0.115108,0.097297,0.040984,0.169811,0.090909
diaghype,0.343291,0.564748,0.383784,0.327869,0.150943,0.272727


#### Fold 0

##### train - overall malignancy = 0.07717391304347826

...

#### Fold 9

##### train - overall malignancy = 0.07694578372527797

race,1.0,2.0,3.0,6.0,4.0,5.0
proportion,0.939245,0.023276,0.018246,0.011934,0.005129,0.002170
count,9523.000000,236.000000,185.000000,121.000000,52.000000,22.000000
label,0.075816,0.110169,0.097297,0.041322,0.173077,0.090909
diaghype,0.343326,0.576271,0.383784,0.330579,0.153846,0.272727


##### valid - overall malignancy = 0.0843263061411549

race,1.0,2.0,6.0,4.0
proportion,0.959633,0.038532,0.000917,0.000917
count,1046.000000,42.000000,1.000000,1.000000
label,0.082218,0.142857,0.000000,0.000000
diaghype,0.342967,0.500000,0.000000,0.000000


## 80% subset

In [12]:
analyze_race_and_malignancy('80')

Theoretical max black proportion = 0.03739989114376798


### Overall dataset:

race,1.0,2.0,3.0,6.0,4.0,5.0
proportion,0.936701,0.029545,0.015201,0.012239,0.004365,0.001949
count,12016.000000,379.000000,195.000000,157.000000,56.000000,25.000000
label,0.076814,0.102902,0.097436,0.038217,0.160714,0.080000
diaghype,0.339643,0.540897,0.400000,0.318471,0.196429,0.240000


#### Fold 0

##### train - overall malignancy = 0.07686328698505916

...

#### Fold 9

##### train - overall malignancy = 0.07741324377852407

race,1.0,2.0,3.0,6.0,4.0,5.0
proportion,0.934139,0.028917,0.016832,0.013466,0.004489,0.002158
count,10822.000000,335.000000,195.000000,156.000000,52.000000,25.000000
label,0.076603,0.101493,0.097436,0.038462,0.173077,0.080000
diaghype,0.339996,0.549254,0.400000,0.320513,0.153846,0.240000


##### valid - overall malignancy = 0.08012820512820513

race,1.0,2.0,4.0,6.0
proportion,0.960579,0.035398,0.003218,0.000805
count,1194.000000,44.000000,4.000000,1.000000
label,0.078727,0.113636,0.000000,0.000000
diaghype,0.336425,0.477273,0.750000,0.000000
